In [325]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timezone
import pytz

Création de la variable d'environnement pour la clé API

In [326]:
load_dotenv()
API_KEY = str(os.getenv('API_KEY'))
print(API_KEY)

847aa628e28d1c21bd1a1cf67c629b6a


Création du Dataframe vide et de la liste des villes

In [327]:
weather_df = pd.DataFrame(columns=["ville", "temp_actuelle", "temp_ressentie", "temp_min", "temp_max", "pression", "humidite", "vitesse_du_vent", "direction_du_vent", "lever_du_soleil", "coucher_du_soleil"])

city_list = ['Paris', 'Lille', 'Marseilles', 'Rouen', 'Bordeaux','Cambrai','Nantes','Metz','Toulouse','Montpellier','Nice','Brest','Arras','Lyon','Strasbourg','Angers','Rennes','Dijon','Grenoble','Caen']


Création d'une boucle pour ajouter les infos de chaque ville au Dataframe et export du Dataframe au format csv

In [328]:
for city in city_list:
    response= requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={API_KEY}")
    temp = response.json()['main']['temp']
    feels_like = response.json()['main']['feels_like']
    temp_min = response.json()['main']['temp_min']
    temp_max = response.json()['main']['temp_max']
    pressure = response.json()['main']['pressure']
    humidity = response.json()['main']['humidity']
    wind_speed = response.json()['wind']['speed']
    wind_deg = response.json()['wind']['deg']
    sunrise = datetime.utcfromtimestamp(response.json()['sys']['sunrise']).strftime('%H:%M:%S')
    sunset = datetime.utcfromtimestamp(response.json()['sys']['sunset']).strftime('%H:%M:%S')
    weather_df = weather_df.append({'ville' : city, 'temp_actuelle' : temp, 'temp_ressentie' : feels_like, 'temp_min': temp_min, 'temp_max': temp_max, 'pression': pressure, 'humidite': humidity, 'vitesse_du_vent': wind_speed, 'direction_du_vent': wind_deg, 'lever_du_soleil': sunrise, 'coucher_du_soleil': sunset}, ignore_index=True)

os.makedirs('./csv', exist_ok=True)  
weather_df.to_csv('./csv/weather.csv', index=False)

/tmp/ipykernel_32137/2897165159.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_df = weather_df.append({'ville' : city, 'temp_actuelle' : temp, 'temp_ressentie' : feels_like, 'temp_min': temp_min, 'temp_max': temp_max, 'pression': pressure, 'humidite': humidity, 'vitesse_du_vent': wind_speed, 'direction_du_vent': wind_deg, 'lever_du_soleil': sunrise, 'coucher_du_soleil': sunset}, ignore_index=True)
/tmp/ipykernel_32137/2897165159.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_df = weather_df.append({'ville' : city, 'temp_actuelle' : temp, 'temp_ressentie' : feels_like, 'temp_min': temp_min, 'temp_max': temp_max, 'pression': pressure, 'humidite': humidity, 'vitesse_du_vent': wind_speed, 'direction_du_vent': wind_deg, 'lever_du_soleil': sunrise, 'coucher_du_soleil': sunset}, ignore_inde

Création de la BDD SQL

In [329]:
import sqlite3

connexion = sqlite3.connect("bdd.db")
curseur = connexion.cursor()

curseur.execute("""
                CREATE TABLE IF NOT EXISTS weather (
                    ville TEXT NOT NULL,
                    temp_actuelle INTEGER NOT NULL,
                    temp_ressentie INTEGER NOT NULL,
                    temp_min INTEGER NOT NULL,
                    temp_max INTEGER NOT NULL,
                    pression INTEGER NOT NULL,
                    humidite INTEGER NOT NULL,
                    vitesse_du_vent INTEGER NOT NULL,
                    direction_du_vent INTEGER NOT NULL,
                    lever_du_soleil DATE NOT NULL,
                    coucher_du_soleil DATE NOT NULL
                )
                """)

for city in city_list:
    response= requests.get(f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={API_KEY}")
    temp = response.json()['main']['temp']
    feels_like = response.json()['main']['feels_like']
    temp_min = response.json()['main']['temp_min']
    temp_max = response.json()['main']['temp_max']
    pressure = response.json()['main']['pressure']
    humidity = response.json()['main']['humidity']
    wind_speed = response.json()['wind']['speed']
    wind_deg = response.json()['wind']['deg']
    sunrise = datetime.utcfromtimestamp(response.json()['sys']['sunrise']).strftime('%H:%M:%S')
    sunset = datetime.utcfromtimestamp(response.json()['sys']['sunset']).strftime('%H:%M:%S')
    curseur.execute("""
                        INSERT INTO weather
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        """, (city, temp, feels_like, temp_min, temp_max, pressure, humidity, wind_speed, wind_deg, sunrise, sunset))

connexion.commit()
connexion.close()